## TFRecord 사용

In [51]:
import os
import tensorflow as tf
import json


# JSON 파일을 읽어 TFExample로 변환하는 함수
def json_to_tfexample(json_data):
    # JSON 데이터 파싱
    metadata = json_data['metaData']
    labeling_info = json_data['labelingInfo']
    
    # TFExample에 넣을 features 설정
    features = {
        'Raw_data_ID': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['Raw data ID'].encode('utf-8')])),
        'breed': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['breed'].encode('utf-8')])),
        'age': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(metadata['age'])])),
        'gender': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['gender'].encode('utf-8')])),
        'species': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['species'].encode('utf-8')])),
        'lesions': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['lesions'].encode('utf-8')])),
        'path': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['Path'].encode('utf-8')])),
        'src_path': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['src_path'].encode('utf-8')]))
    }
    

    # Polygon 정보 추출
    polygon_location = labeling_info[0]['polygon']['location'][0]

    # x, y 좌표를 따로 리스트로 저장
    x_coords = [polygon_location[f'x{i}'] for i in range(1, len(polygon_location) // 2 + 1)]
    y_coords = [polygon_location[f'y{i}'] for i in range(1, len(polygon_location) // 2 + 1)]

    # TFRecord의 feature 업데이트
    polygon_features = {
        f'x{i}': tf.train.Feature(int64_list=tf.train.Int64List(value=[x_coords[i - 1]])) for i in range(1, len(x_coords) + 1)
    }
    polygon_features.update({
        f'y{i}': tf.train.Feature(int64_list=tf.train.Int64List(value=[y_coords[i - 1]])) for i in range(1, len(y_coords) + 1)
    })

    # Label 업데이트
    label = labeling_info[0]['polygon']['label']
    features.update(polygon_features)
    features['label'] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[label.encode('utf-8')]))



    # Box 정보 추출
    if any('box' in info for info in labeling_info):
        for info in labeling_info:
            if 'box' in info:
                box_data = info['box']
                box_location = box_data['location'][0] if box_data['location'] else {}  # 빈 딕셔너리로 처리
                box_features = {
                    'box_x': tf.train.Feature(int64_list=tf.train.Int64List(value=[box_location.get('x', -1)])),
                    'box_y': tf.train.Feature(int64_list=tf.train.Int64List(value=[box_location.get('y', -1)])),
                    'box_width': tf.train.Feature(int64_list=tf.train.Int64List(value=[box_location.get('width', -1)])),
                    'box_height': tf.train.Feature(int64_list=tf.train.Int64List(value=[box_location.get('height', -1)]))
                }
                features.update(box_features)
                features['box_label'] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[box_data.get('label', '').encode('utf-8')]))
    else:
        print(f"Box 정보가 없는 JSON 파일: {metadata['Raw data ID']}")

    example = tf.train.Example(features=tf.train.Features(feature=features))
    return example

# TFRecord 파일로 변환하는 함수
def convert_to_tfrecord(folder_path, output_file):
    json_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))

    with tf.io.TFRecordWriter(output_file) as writer:
        for json_file in json_files:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            example = json_to_tfexample(data)
            writer.write(example.SerializeToString())

# JSON 파일이 있는 폴더 경로
folder_path = r'D:\152_pet_dermatological_data\01_data\2_Validation\2_labeling_data\VL01\반려견\피부\일반카메라\유증상\A1_구진_플라크'

# TFRecord 파일 경로
output_file = 'val_symptoms_A1.tfrecord'

# TFRecord 파일로 변환
convert_to_tfrecord(folder_path, output_file)

print(f'TFRecord 파일이 생성되었습니다: {output_file}')


TFRecord 파일이 생성되었습니다: val_symptoms_A1.tfrecord


In [52]:
import tensorflow as tf
import pandas as pd

# TFRecord 파일을 읽고 데이터 프레임으로 변환하는 함수 정의
def tfrecord_to_dataframe(tfrecord_file):
    # TFRecordDataset을 사용하여 TFRecord 파일 읽기
    dataset = tf.data.TFRecordDataset(tfrecord_file)

    # TFRecord 파일 내용을 저장할 리스트 초기화
    records = []

    # TFRecord 파일 내용 읽기
    for record in dataset:
        example = tf.train.Example()
        example.ParseFromString(record.numpy())

        # Example 내용을 딕셔너리로 변환하여 리스트에 추가
        record_dict = {}
        for key in example.features.feature:
            # TFExample의 각 feature에 대해 바이트 리스트 값을 추출
            feature = example.features.feature[key]
            # 값이 있는 경우에만 딕셔너리에 추가
            if feature.HasField('bytes_list'):
                record_dict[key] = feature.bytes_list.value[0].decode('utf-8')
            elif feature.HasField('int64_list'):
                record_dict[key] = feature.int64_list.value[0]
            elif feature.HasField('float_list'):
                record_dict[key] = feature.float_list.value[0]
            # 그 외의 경우는 None으로 설정
            else:
                record_dict[key] = None
        
        records.append(record_dict)

    # 리스트를 데이터 프레임으로 변환
    df = pd.DataFrame(records)
    return df

# TFRecord 파일 경로
tfrecord_file = 'val_symptoms_A1.tfrecord'

# TFRecord 파일을 데이터 프레임으로 변환
df = tfrecord_to_dataframe(tfrecord_file)

# 데이터 프레임 확인
df.to_excel('val_symptoms_A1.xlsx')
